In [7]:
# noexport

!typech evaluate_model.ipynb

[NbConvertApp] Converting notebook evaluate_model.ipynb to python
evaluate_model.py:120: error: Need type annotation for 'all_losses'


In [1]:
from mkdata import *

In [3]:
#print(get_num_features(get_feature_names()))

In [2]:
import model_selfattentionlstm
reload(model_selfattentionlstm)
from model_selfattentionlstm import SelfAttentionLSTM

In [3]:
import os
import torch
import math

In [6]:
all_features_data = get_all_features_data()

exception in jsonmemo for file 2019_04_08/get_all_features_data.json
[Errno 2] No such file or directory: '2019_04_08/get_all_features_data.json'
performing computation 2019_04_08/get_all_features_data.json
exception in jsonmemo for file 2019_04_08/get_users.json
[Errno 2] No such file or directory: '2019_04_08/get_users.json'
performing computation 2019_04_08/get_users.json
done with computation 2019_04_08/get_users.json
e0ea34c81d4b50cddc7bd752
d45b8b82dec8ec3c591fdbd7
f4d95006c663a799d9185576
8d2c9eb27dee2dc85bca705b
4a5f492c987b873871be4c12
78cc8c70b8be7412e00a1575
04c7fb2ae3098eab5ae390b8
8642884bd204eafc660a04a2
9ca53f1f33ac3e25c2e3f3d2
6e055258e3625ab1db57f0fa
9c00827409f8c296d67f78b4
687606ae6354d8b707f064ee
57ddb2820db2de361757d21b
af8454409cde73bead9d1071
eccfe39b9019a3f566346399
f32cf25130f6fa751892960e
8f77439cf19de38abfd4a417
3bc557eda64de5ff9d2e6fca
16043b6d098957d10448c009
edc3437cabf565d7a8c2f96c
88550c3144f8e1a6e84e83e2
3bc201cd34fc67185861245e
472a909dcf35ed6f1bc6da82

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



44c348106c62c885c6b3effa
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-cabf191a73e1>", line 1, in <module>
    all_features_data = get_all_features_data()
  File "/home/geza/motivation/jsonmemo.py", line 72, in wrapped
    cache = f()
  File "/home/geza/motivation/mkdata.py", line 324, in get_all_features_data
    print(user)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/iostream.py", line 400, in write
    self.pub_thread.schedule(lambda : self._buffer.write(string))
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/iostream.py", line 203, in schedule
    self._event_pipe.send(b'')
  File "/usr/local/lib/python3.7/dist-packages/zmq/sugar/socket.py", line 395, in send
    return super(Socket, self).send(data, flags=flags, copy=copy, track=track)
  File "zmq/backend/cython/socket.pyx", line 725, in 

KeyboardInterrupt: 

In [7]:
training_data_all_features,dev_data_all_features,test_data_all_features = split_into_train_dev_test(all_features_data)

In [8]:
feature_names = get_feature_names()
num_prior_entries = 10


learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

dev_data = make_tensors_from_features(dev_data_all_features, {'enabled_feature_list': feature_names, 'num_prior_entries': num_prior_entries})
training_data = make_tensors_from_features(training_data_all_features, {'enabled_feature_list': feature_names, 'num_prior_entries': num_prior_entries})


In [9]:
def categoryFromOutput(output):
  top_n,top_i = output.topk(1)
  category_i = top_i[0].item()
  return ['nothing','easy','medium','hard'][category_i],category_i
#print(output.topk(1))
#print(categoryFromOutput(output))

In [10]:
def evaluate3(model, line_tensor):
    line_tensor = line_tensor.permute(1,0,2)
    #model.zero_grad()
    
    # TODO: construct mask and lengths for batch size == 1
    lengths = torch.tensor([line_tensor.size()[0]])
    mask = torch.zeros(line_tensor.size()[0], line_tensor.size()[1], 1, dtype=torch.float)
    device = (torch.device('cuda') if torch.cuda.is_available() else
                   torch.device('cpu'))
    mask = mask.to(device)
    line_tensor = line_tensor.to(device)
    #category_tensor = category_tensor.to(device)
    # END TODO
#     print(line_tensor.size())
#     print(category_tensor.size())
#     print(lengths.size())
#     print(mask.size())
    output = model(line_tensor, lengths, mask)
    # print(category_tensor.size())
    #loss = criterion(output, category_tensor)
    #loss.backward()
    return output

In [11]:
def evaluate_model_on_dataset(model, dataset):
  num_correct = 0
  num_incorrect = 0
  for item in dataset:
    category_tensor = item['category']
    feature_tensor = item['feature']
    if feature_tensor.size()[0] == 0:
        continue
    difficulty = tensor_to_difficulty(category_tensor)
    difficulty_idx = get_difficulty_idx(difficulty)
    predicted_tensor = evaluate3(model, feature_tensor.cuda())
    predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
    if predicted_difficulty_idx == difficulty_idx:
      num_correct += 1
    else:
      num_incorrect += 1
    #confusion[difficulty_idx][predicted_difficulty_idx] += 1
    #print(predicted_difficulty)
    #print(difficulty)
    #break

  print('num correct: ' + str(num_correct))
  print('num incorrect: ' + str(num_incorrect))
  print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))

In [12]:
def tensor_to_difficulty(tensor):
  difficulty_idx = tensor[0].data.cpu().numpy()
  return ['nothing', 'easy', 'medium', 'hard'][difficulty_idx]

print(make_tensor_from_chosen_difficulty('hard'))
print(tensor_to_difficulty((make_tensor_from_chosen_difficulty('hard'))))

tensor([3])
hard


In [15]:
n_iters = 100
print_every = 1
plot_every = 1
all_losses = []
num_features = get_num_features(get_feature_names())

for sample_every_n_visits in range(1, 11):
  print('making training data for sample_every_n_visits', sample_every_n_visits)
  #training_data = make_tensors_from_features(training_data_all_features[:int(len(training_data_all_features) / 100)], {
  #  'enabled_feature_list': feature_names,
  #  'num_prior_entries': num_prior_entries,
  #  'enable_current_difficulty': enable_current_difficulty,
  #  'sample_every_n_visits': sample_every_n_visits,
  #})
  model = SelfAttentionLSTM({'word_embed_size': num_features, 'window_embed_size': 128})
  model_filename = 'model_attention_nohistory_fractiondata10_nhidden512_sample_every_n_visits_' + str(sample_every_n_visits) + '_v10_epoch1.pt'
  model_state = torch.load(model_filename)
  model.load_state_dict(model_state['model_state_dict'])
  model.eval()
  evaluate_model_on_dataset(model, dev_data)


making training data for sample_every_n_visits 1
num correct: 3810
num incorrect: 1762
percentage correct: 0.6837760229720029
making training data for sample_every_n_visits 2
num correct: 3997
num incorrect: 1575
percentage correct: 0.7173366834170855
making training data for sample_every_n_visits 3
num correct: 3988
num incorrect: 1584
percentage correct: 0.715721464465183
making training data for sample_every_n_visits 4
num correct: 3982
num incorrect: 1590
percentage correct: 0.7146446518305815
making training data for sample_every_n_visits 5


FileNotFoundError: [Errno 2] No such file or directory: 'model_attention_nohistory_fractiondata10_nhidden512_sample_every_n_visits_5_v10_epoch1.pt'

In [30]:
parameters = {
  'word_embed_size': 277,
  'window_embed_size': 128,
}
epoch = 0
while True:
  epoch += 1
  model_filename = 'model_attention_nohistory_nhidden512_v10_epoch' + str(epoch) + '.pt'
  if not os.path.isfile(model_filename):
    break
  print(model_filename)
  model = SelfAttentionLSTM(parameters)
  model_state = torch.load(model_filename)
  model.load_state_dict(model_state['model_state_dict'])
  model.eval()
  evaluate_model_on_dataset(model, dev_data)
  #break


model_attention_nohistory_nhidden512_v10_epoch1.pt
num correct: 5023
num incorrect: 461
percentage correct: 0.9159372720641867
model_attention_nohistory_nhidden512_v10_epoch2.pt
num correct: 5001
num incorrect: 483
percentage correct: 0.911925601750547
model_attention_nohistory_nhidden512_v10_epoch3.pt
num correct: 5011
num incorrect: 473
percentage correct: 0.9137490882567469
model_attention_nohistory_nhidden512_v10_epoch4.pt
num correct: 5035
num incorrect: 449
percentage correct: 0.9181254558716265
model_attention_nohistory_nhidden512_v10_epoch5.pt
num correct: 5032
num incorrect: 452
percentage correct: 0.9175784099197666
model_attention_nohistory_nhidden512_v10_epoch6.pt
num correct: 5031
num incorrect: 453
percentage correct: 0.9173960612691466
model_attention_nohistory_nhidden512_v10_epoch7.pt


KeyboardInterrupt: 

In [34]:
parameters = {
  'word_embed_size': 277,
  'window_embed_size': 128,
}
epoch = 0
while True:
  epoch += 1
  model_filename = 'model_attention_nohistory_nhidden512_v10_epoch' + str(epoch) + '.pt'
  if not os.path.isfile(model_filename):
    break
  print(model_filename)
  model = SelfAttentionLSTM(parameters)
  model_state = torch.load(model_filename)
  model.load_state_dict(model_state['model_state_dict'])
  model.eval()
  evaluate_model_on_dataset(model, training_data)
  #break


model_attention_nohistory_nhidden512_v10_epoch1.pt
num correct: 90795
num incorrect: 7911
percentage correct: 0.9198528964804571
model_attention_nohistory_nhidden512_v10_epoch2.pt
num correct: 90866
num incorrect: 7840
percentage correct: 0.9205722043239519
model_attention_nohistory_nhidden512_v10_epoch3.pt


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-34-c6aea99d77f9>", line 16, in <module>
    evaluate_model_on_dataset(model, training_data)
  File "<ipython-input-29-e020bc8d25b5>", line 11, in evaluate_model_on_dataset
    predicted_tensor = evaluate3(model, feature_tensor.cuda())
  File "<ipython-input-18-da8906b84de7>", line 10, in evaluate3
    mask = mask.to(device)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
 

KeyboardInterrupt: 